In [1]:
import os

import torch
from matplotlib import pyplot as plt

from utils import plot_ci

In [2]:
input_lang = 'zh'
target_lang = 'fr'
model_size = '7b'
out_dir = './visuals'
save_dir = '../eval_resuslts/logit_lens/summaries/'
save_plots = True
save_format = ".png"
# save_format = ".pdf"
# target_task = "translation"
target_task = "cloze"

In [3]:
latents = []
outputs = []

langs_infix = f"{input_lang}_{target_lang}" if target_task == "translation" else f"{target_lang}"

def load_latents(model_label):
    return [torch.load(f'{os.path.join(out_dir, model_label)}/{target_task}/{model_label}_{langs_infix}_latent_probs.pt')]

def load_outputs(model_label):
    return [torch.load(f'{os.path.join(out_dir, model_label)}/{target_task}/{model_label}_{langs_infix}_out_probs.pt')]


source_labels = [
    "hf_16bit",
    "bnb_4bit",
    "rtn_4bit",
    "awq_4bit",
    "gptq_4bit",
    # "twq_4bit_combined",
    # "twq_4bit_random",
    # "mpq_4bit_combined",
    # "mpq_4bit_random",
    # "mpqr_4bit_combined",
    # "mpqr_4bit_random",
]

for label in source_labels:
    latents += load_latents(label)
    outputs += load_outputs(label)

In [4]:
size2tik = {"tiny" : 5, '7b': 5, '13b': 5, '70b': 10}


def plot_custom(ax, output, label, color='black', delta=0, confidence=False, marker=None, do_lines=False, offset=0, line_alpha=1):
    plot_ci(ax, (output-delta)[:,offset:], label, color, do_lines=do_lines, plt_params={"linewidth":1, "marker":marker, "alpha":line_alpha}, do_confidence=confidence, layer_idx_offset=offset, confidence_alpha=0.1)

In [5]:
label_base = [
    "base",
    "bnb 4bit",
    "rtn 4bit",
    "awq 4bit",
    "gptq 4bit",
    # "twq 4bit\n(lang. neurons)"
    # "twq 4bit\n(random)"
    # "mpq 4bit\n(lang. neurons)"
    # "mpq 4bit\n(random)"
    # "mpqr 4bit\n(lang. neurons)"
    # "mpqr 4bit\n(random)"
]

In [ ]:
fig, ax2 = plt.subplots(figsize=(10, 7))

plt.axhline(y=0, color='grey', linestyle='--', linewidth=0.7, label="baseline")

lang = target_lang

offset = 17
delta = outputs[0]
labels = [lang + " " + lab for lab in label_base]
cmap = plt.get_cmap('turbo')
colors = [cmap(i / (len(outputs) - 1)) for i in range(len(outputs))]
markers = ["*"] + ["o"]*(len(outputs)-1)
for n in range(1, len(outputs)):
    plot_custom(ax2, outputs[n], labels[n], color=colors[n], marker=markers[n] ,delta=delta, offset=offset)


ax2.set_xlabel('layer')
ax2.set_ylabel('probability delta')
if model_size == '7b' or "tiny":
    ax2.set_xlim(offset, outputs[0].shape[1]+1)
else:
    ax2.set_xlim(offset, round(outputs[0].shape[1]/10)*10+1)

ax2.set_ylim(-0.35, 0.1)
# ax2.set_yscale('log')

# put legend on the top left
ax2.set_title(f"Probability delta of the [{lang}] token response")
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
if save_plots:
    os.makedirs(f'{save_dir}/{target_task}/{langs_infix}', exist_ok=True)
    plt.savefig(f'{save_dir}/{target_task}/{langs_infix}/{model_size}_{langs_infix}_probas_ent_tgt_delta{save_format}', dpi=300, bbox_inches='tight')

In [ ]:
fig, ax2 = plt.subplots(figsize=(10, 7))

lang = target_lang

offset = 17
delta = 0
labels = [lang + " " + lab for lab in label_base]
cmap = plt.get_cmap('turbo')
colors = [cmap(i / (len(outputs) - 1)) for i in range(len(outputs))]
markers = ["*"] + ["o"]*(len(outputs)-1)
for n in range(0, len(outputs)):
    plot_custom(ax2, outputs[n], labels[n], color=colors[n], marker=markers[n] ,delta=delta, offset=offset)


ax2.set_xlabel('layer')
ax2.set_ylabel('probability')
if model_size == '7b' or "tiny":
    ax2.set_xlim(offset, outputs[0].shape[1]+1)
else:
    ax2.set_xlim(offset, round(outputs[0].shape[1]/10)*10+1)

ax2.set_ylim(0, 0.8)


ax2.set_title(f"Probability of the [{lang}] token response")
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
if save_plots:
    os.makedirs(f'{save_dir}/{target_task}/{langs_infix}', exist_ok=True)
    plt.savefig(f'{save_dir}/{target_task}/{langs_infix}/{model_size}_{langs_infix}_probas_ent_tgt{save_format}', dpi=300, bbox_inches='tight')

In [ ]:
fig, ax2 = plt.subplots(figsize=(10, 7))

lang = "en"

offset = 15
delta = 0#latents[0]
labels = [lang + " " + lab for lab in label_base]
cmap = plt.get_cmap('turbo')
colors = [cmap(i / (len(outputs) - 1)) for i in range(len(outputs))]
markers = ["*"] + ["o"]*(len(outputs)-1)
for n in range(0, len(latents)):
    plot_custom(ax2, latents[n], labels[n], color=colors[n], marker=markers[n] ,delta=delta, offset=offset)


ax2.set_xlabel('layer')
ax2.set_ylabel('probability')
if model_size == '7b' or "tiny":
    ax2.set_xlim(offset, latents[0].shape[1]+1)
else:
    ax2.set_xlim(offset, round(latents[0].shape[1]/10)*10+1)

ax2.set_title(f"Probability of the [{lang}] token response")
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
if save_plots:
    os.makedirs(f'{save_dir}/{target_task}/{langs_infix}', exist_ok=True)
    plt.savefig(f'{save_dir}/{target_task}/{langs_infix}/{model_size}_{langs_infix}_probas_ent_src{save_format}', dpi=300, bbox_inches='tight')

In [ ]:
fig, ax2 = plt.subplots(figsize=(10, 7))

plt.axhline(y=0, color='grey', linestyle='--', linewidth=0.7, label="baseline")

lang = "en"

offset = 15
delta = latents[0]
labels = [lang + " " + lab for lab in label_base]
cmap = plt.get_cmap('turbo')
colors = [cmap(i / (len(outputs) - 1)) for i in range(len(outputs))]
markers = ["*"] + ["o"]*(len(outputs)-1)
for n in range(1, len(outputs)):
    plot_custom(ax2, latents[n], labels[n], color=colors[n], marker=markers[n] ,delta=delta, offset=offset)


ax2.set_xlabel('layer')
ax2.set_ylabel('probability delta')
if model_size == '7b' or "tiny":
    ax2.set_xlim(offset, latents[0].shape[1]+1)
else:
    ax2.set_xlim(offset, round(latents[0].shape[1]/10)*10+1)

ax2.set_title(f"Probability delta of the [{lang}] token response")
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
if save_plots:
    os.makedirs(f'{save_dir}/{target_task}/{langs_infix}', exist_ok=True)
    plt.savefig(f'{save_dir}/{target_task}/{langs_infix}/{model_size}_{langs_infix}_probas_ent_src_delta{save_format}', dpi=300, bbox_inches='tight')

In [ ]:
fig, ax2 = plt.subplots(figsize=(10, 7))


offset = 17
delta = 0#latents[0]

selected_indeces = range(len(outputs))
idx_end = len(selected_indeces)

cmap = plt.get_cmap('turbo')
colors = [cmap(i / (idx_end*2 - 1)) for i in range(idx_end*2+1)]
colors[idx_end] = colors[0]

lang = "en"
labels = [lang + " " + lab for lab in label_base]
markers = ["*"] + ["s"]*len(latents)
for n, m in zip(selected_indeces, range(idx_end)):
    plot_custom(ax2, latents[n], labels[n], color=colors[:idx_end][m], marker=markers[n] ,delta=delta, offset=offset, confidence=True)


lang = target_lang
labels = [lang + " " + lab for lab in label_base]
markers = ["*"] + ["o"]*len(latents)

for n, m in zip(selected_indeces, range(idx_end)):
    plot_custom(ax2, outputs[n], labels[n], color=colors[idx_end:][m], marker=markers[n] ,delta=delta, offset=offset, confidence=True)

ax2.set_xlabel('layer')
ax2.set_ylabel('probability')
if model_size == '7b' or "tiny":
    ax2.set_xlim(offset, latents[0].shape[1]+1)
else:
    ax2.set_xlim(offset, round(latents[0].shape[1]/10)*10+1)

ax2.set_title(f"Probability of token responses in [{target_lang}] and [en]")
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
if save_plots:
    os.makedirs(f'{save_dir}/{target_task}/{langs_infix}', exist_ok=True)
    plt.savefig(f'{save_dir}/{target_task}/{langs_infix}/{model_size}_{langs_infix}_probas_ent_comp{save_format}', dpi=300, bbox_inches='tight')

In [ ]:
fig, ax2 = plt.subplots(figsize=(10, 7))

plt.axhline(y=0, color='grey', linestyle='--', linewidth=0.7, label="baseline")

offset = 17
delta = latents[0]

selected_indeces = range(len(outputs))
idx_end = len(selected_indeces)

cmap = plt.get_cmap('turbo')
colors = [cmap(i / (idx_end*2 - 1)) for i in range(idx_end*2+1)]
# colors[idx_end] = colors[0]

lang = "en"
labels = [lang + " " + lab for lab in label_base]
markers = ["*"] + ["s"]*len(latents)
for n, m in zip(selected_indeces, range(idx_end)):
    plot_custom(ax2, latents[n], labels[n], color=colors[:idx_end][m], marker=markers[n] ,delta=delta, offset=offset, confidence=True)

delta = outputs[0]

lang = target_lang
labels = [lang + " " + lab for lab in label_base]
markers = ["*"] + ["o"]*len(latents)

for n, m in zip(selected_indeces, range(idx_end)):
    plot_custom(ax2, outputs[n], labels[n], color=colors[idx_end:][m], marker=markers[n] ,delta=delta, offset=offset, confidence=True)

ax2.set_xlabel('layer')
ax2.set_ylabel('probability delta')
if model_size == '7b' or "tiny":
    ax2.set_xlim(offset, latents[0].shape[1]+1)
else:
    ax2.set_xlim(offset, round(latents[0].shape[1]/10)*10+1)

ax2.set_title(f"Probability delta of token responses in [{target_lang}] and [en]")
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
if save_plots:
    os.makedirs(f'{save_dir}/{target_task}/{langs_infix}', exist_ok=True)
    plt.savefig(f'{save_dir}/{target_task}/{langs_infix}/{model_size}_{langs_infix}_probas_ent_comp_deltas{save_format}', dpi=300, bbox_inches='tight')

In [ ]:
fig, ax2 = plt.subplots(figsize=(10, 7))

plt.axhline(y=0, color='grey', linestyle='--', linewidth=0.7, label="baseline")

offset = 17
delta = latents[0]

selected_indeces = range(len(outputs))
idx_end = len(selected_indeces)

cmap = plt.get_cmap('turbo')
colors = [cmap(i / (idx_end*2 - 1)) for i in range(idx_end*2+1)]
colors[idx_end] = colors[0]

lang = "en"
labels = [lang + " " + lab for lab in label_base]
markers = ["*"] + ["s"]*len(latents)
for n, m in zip(selected_indeces, range(idx_end)):
    plot_custom(ax2, latents[n], labels[n], color=colors[:idx_end][m], marker=markers[n] ,delta=delta, offset=offset, confidence=True)

delta = outputs[0]

lang = target_lang
labels = [lang + " " + lab for lab in label_base]
markers = ["*"] + ["o"]*len(latents)

for n, m in zip(selected_indeces, range(idx_end)):
    plot_custom(ax2, outputs[n], labels[n], color=colors[idx_end:][m], marker=markers[n] ,delta=delta, offset=offset, confidence=True)

ax2.set_xlabel('layer')
ax2.set_ylabel('probability delta')
if model_size == '7b' or "tiny":
    ax2.set_xlim(offset, latents[0].shape[1]+1)
else:
    ax2.set_xlim(offset, round(latents[0].shape[1]/10)*10+1)

ax2.set_title(f"Normalized probability delta of token responses\nwhen translating from [{input_lang}] to [{target_lang}], in [{target_lang}] and [en]")
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
if save_plots:
    os.makedirs(f'{save_dir}/{target_task}/{langs_infix}', exist_ok=True)
    plt.savefig(f'{save_dir}/{target_task}/{langs_infix}/{model_size}_{langs_infix}_probas_ent_comp_deltas{save_format}', dpi=300, bbox_inches='tight')

In [ ]:
fig, ax2 = plt.subplots(figsize=(10, 7))

lang = ""

plt.axhline(y=0, color='grey', linestyle='--', linewidth=0.7)

offset = 15
delta = 0#outputs[0]
labels = label_base
cmap = plt.get_cmap('turbo')
colors = [cmap(i / (len(outputs) - 1)) for i in range(len(outputs))]
markers = ["*"] + ["o"]*(len(outputs)-1)
for n in range(0, len(outputs)):
    plot_custom(ax2, outputs[n], labels[n], color=colors[n], marker=markers[n] ,delta=latents[n], offset=offset, line_alpha=0.7)


ax2.set_xlabel('layer')
ax2.set_ylabel('probability delta')
if model_size == '7b' or "tiny":
    ax2.set_xlim(offset, outputs[0].shape[1]+1)
else:
    ax2.set_xlim(offset, round(outputs[0].shape[1]/10)*10+1)

ax2.set_title(f"Probability difference between [{target_lang}] and [en] token responses (bigger is better)")
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
if save_plots:
    os.makedirs(f'{save_dir}/{target_task}/{langs_infix}', exist_ok=True)
    plt.savefig(f'{save_dir}/{target_task}/{langs_infix}/{model_size}_{langs_infix}_probas_compare{save_format}', dpi=300, bbox_inches='tight')